<a href="https://colab.research.google.com/github/Songhyunjeong/thingplug-lora-starter-kit/blob/master/notebookd5c4df25c8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'postpia4:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4116903%2F7134966%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240226%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240226T084303Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5c0c4dba68abc21dd87507b78bb2345deaf469af9b0a155379ec8e85677f1ad73168da523a57e2f894f9c5f6f40358244907684c983a20584deb457e23e6f40b21d26d87340e3b284e33f81b479b4dac97bcca604ad5d8dbfd246cedd0e08a1535c51e97d2cfbb67e3579690e365ddb1be797f6f62f8aa69e3b6d61e028ec13a56966a30024f457d376940c36fb357be3e038710d9d9a5f5815bad2f2392b0270fd713bfad16a52a19ea5836a3f6f38c90febcd0ea24f9eb160d7b2ab50d893b4f3380195ebaed2398f4649b5d43bca3cbc779fc20eb43fb6382309367afe08bbe0dfcb987f44a06a8245af9f3e13d1f8de575f95d7b178b1d3e6ce14500e738,basic-01:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4483020%2F7683464%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240226%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240226T084303Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D02af107f31b3673a81e364ed7d720a4ed6078b2b64bf13015ea17a0f40b2a59111cd2427268bbef627d77f23aedcc97144c5264db22763fd215936640f92eb94db398767ff880a2eba3955c959942d53b98a0a05ee52f46c7378614a29533e3cf5a36f804a6e50b428ae64b90a6b62bffa7b870bc9a8f215c2da4c8354b1608f1673776fd0fc42564828da527c455bd6d68f2af91805f8d789e2a60943d430f10f583c58b04f0222b164840b03cd6e5429dacbff9b27ef9f36d45efe5c3f7911c403d3a9577b4e651adc3ddd497cd2e307f289c71d7504496756232649520e8e684407f815c144ed702c59ea1dbf411134315e535d700d54865a789e79c4bd02,data-02:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4497059%2F7703368%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240226%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240226T084303Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D45eae022cd0bdffb519b71cbeeabf80e4905863d70c6fe8d79eae8e01518e12d0b24e91d4951b12f15e7474ff5cc5f2003ca017a7d8975cffc531b0caa17efbca79bcf09efd6ab5a7045efadb694d9660c9654d0d2e55f50eef9caf59fcdfc6046b3b72846318cef64b5190a788c5f008cc3d37a22fcedd3a672c322070b24366fce2d1e7295e5fcbb6fe77875df4fc46005994f70853918f6c64e83dcb1938397067e78c56857401483951f8d18b74877f1e46f40c5a2c4cb614bd269fd0585e8aaee576999f9020bfd3e9de2637d8dee496f1c727c7e527ae5e88282f116de434778eba8ed6cebe5cf5285307e26e9f7376ad5db42da2fb90f5171b2216e5c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1193429 bytes downloaded
Downloaded and uncompressed: postpia4
[==================================================] 5253153 bytes downloaded
Downloaded and uncompressed: basic-01
[==================================================] 6831306 bytes downloaded
Downloaded and uncompressed: data-02
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-02/data.pdf
/kaggle/input/postpia4/opendata.pdf
/kaggle/input/basic-01/basic_manual.pdf


아래 코드를 차례대로 실행하거나 Run All 실행하세요.원하는 파일(PDF)을 업로드하여 변경 적용해주시면 됩니다.샘플로 보시고 업무에 적용해 보시면 좋을 듯합니다.(카피해서 새로운 노트북 생성해서 사용하셔도 됩니다)
키값은 유료계정으로 발급받은 키값이고 현재 약 20달러 충전되어 있습니다. 많이 사용해도 얼마 차감안되니 마음껏 사용하셔도 됩니다.

In [ ]:
!pip install -U langchain-openai

!pip install openai
!pip install chromadb
!pip install tiktoken

In [ ]:
from google.colab import userdata
a=userdata.get('Secret_key')

from dotenv import load_dotenv
load_dotenv()

# PyPDFLoader
from langchain.document_loaders import PyPDFLoader

# TextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Loader
loader = PyPDFLoader("/kaggle/input/basic-01/basic_manual.pdf")
pages = loader.load_and_split()




# Split
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 300,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_documents(pages)
print(texts[0])

# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(openai_api_key="a")

# load it into Chroma
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings_model)

# Question & Answer
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key="a")
qa_chain = RetrievalQA.from_chain_type(llm,retriever=db.as_retriever())




In [ ]:
!pip install pypdf
!pip install langchain

In [ ]:

my_dict = {}


while True:
    # 사용자로부터 질문 입력 받기
    question = input("질문을 입력하세요 (종료하려면 '종료'를 입력하세요): ")

    # '종료' 입력 시 루프 종료
    if question == '종료':
        break

    # 답변 생성 (여기서는 간단히 예시로 "질문에 대한 답변"으로 대체)
    answer = qa_chain({"query": question})

    # 딕셔너리에 질문과 답변 추가
    my_dict[question] = answer
    print(answer)


with open("질문과_답변.txt", "w", encoding="utf-8") as file:
    for question, answer in my_dict.items():
        file.write(f"질문: {question}\n")
        file.write(f"답변: {answer}\n\n")

print("질문과 답변이 파일로 저장되었습니다.")

In [ ]:
!pip install openai

In [ ]:
python --version
